In [ ]:
import pandas as pd
import collections
import numpy as np
from sklearn import set_config
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

In [ ]:
# downloading the dataset

import gdown
url = 'https://drive.google.com/uc?id=1JQ_gVoF1z57Zo2U_mDBTsOUKsWc95G2w'
data_tsv = 'data.tsv'
gdown.download(url, data_tsv, quiet=False)
data_file = 'data.tsv'

Downloading...
From: https://drive.google.com/uc?id=1JQ_gVoF1z57Zo2U_mDBTsOUKsWc95G2w
To: /content/data.tsv
100%|██████████| 106k/106k [00:00<00:00, 70.7MB/s]


In [ ]:
# initializing the dataframe 

import  pandas as pd
df0 = pd.read_csv(data_file, sep="\t")

In [ ]:
df = df0.copy(deep=True)

In [ ]:
df = df.iloc[:,1:]

In [ ]:
df.head()

,Group,Sex,Age,Patients number per hour,Arrival mode,Injury,Cheif_Complain_Translated,Mental,Pain,NRS_pain,SBP,DBP,HR,RR,BT,Saturation,Diagnosis in ED,KTAS_expert
0,2,2,71,3,3,2,right ocular pain,1,1,2.0,160.0,100.0,84.0,18.0,36.6,100.0,Corneal abrasion,4
1,1,1,56,12,3,2,right forearm burn,1,1,2.0,137.0,75.0,60.0,20.0,36.5,NaN,"Burn of hand, firts degree dorsum",5
2,2,1,68,8,2,2,"arm pain, Lt",1,1,2.0,130.0,80.0,102.0,20.0,36.6,98.0,"Fracture of surgical neck of humerus, closed",5
3,1,2,71,8,1,1,ascites tapping,1,1,3.0,139.0,94.0,88.0,20.0,36.5,NaN,Alcoholic liver cirrhosis with ascites,5
4,1,2,58,4,3,1,"Distension, Abd",1,1,3.0,91.0,67.0,93.0,18.0,36.5,NaN,Ascites,5


In [ ]:
# Arrival modes '5' and '7' had very low occurences (2 each) 
# so those rows were dropped to reduce feature size

df = df[df['Arrival mode'] != 5]
df = df[df['Arrival mode'] != 7]

In [ ]:
# Replacing NaN values in text features with empty strings

df['Diagnosis in ED'].replace(np.nan, '', inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:4582: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


In [ ]:
df.isnull().sum()

Group                          0
Sex                            0
Age                            0
Patients number per hour       0
Arrival mode                   0
Injury                         0
Cheif_Complain_Translated      0
Mental                         0
Pain                           0
NRS_pain                     552
SBP                           25
DBP                           29
HR                            20
RR                            22
BT                            18
Saturation                   694
Diagnosis in ED                0
KTAS_expert                    0
dtype: int64

In [ ]:
# feature-target split 

X, y = df.drop(columns=['KTAS_expert'], axis=1), df['KTAS_expert']

In [ ]:
# train-test split

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.33, 
                                                    random_state=30) 

In [ ]:
feat_impute_0 = ['NRS_pain']                                            # features where NaN values have to be replaed with 0
feat_impute_mean = ['SBP', 'DBP', 'HR', 'RR', 'BT', 'Saturation']       # features where NaN values have to be replaed with mean
feat_onehot = ['Group', 'Sex', 'Arrival mode', 'Injury', 'Pain']        # features which have to be one-hot encoded
feat_scale_rest = ['Age', 'Patients number per hour', 'Mental']         # rest of the features which only have to be normalized

In [ ]:
simple_imputer_0 = SimpleImputer(strategy='constant', fill_value=0)     
simple_imputer_0.fit(X[feat_impute_0])
standard_scaler_impute_0 = StandardScaler()
standard_scaler_impute_0.fit(simple_imputer_0.transform(X[feat_impute_0]))

pipe_impute_0 = Pipeline([                                              # 0 imputer and normalizing pipeline
    ('imputer', simple_imputer_0),
    ('scaler', standard_scaler_impute_0)
])

In [ ]:
simple_imputer_mean = SimpleImputer()
simple_imputer_mean.fit(X[feat_impute_mean])
standard_scaler_impute_mean = StandardScaler()
standard_scaler_impute_mean.fit(simple_imputer_mean.transform(X[feat_impute_mean]))

pipe_impute_mean = Pipeline([                                           # mean imputer and normalizing pipeline
    ('imputer', simple_imputer_mean),
    ('scaler', standard_scaler_impute_mean)
])

In [ ]:
count_vectorizer_complain = CountVectorizer()   
count_vectorizer_complain.fit(X['Cheif_Complain_Translated'])
lsa_complain = TruncatedSVD(n_components=120)
lsa_complain.fit(count_vectorizer_complain.transform(X['Cheif_Complain_Translated']))

pipe_complain = Pipeline([                                              # text feature vectorizer and decomposition pipeline
    ('cvec', count_vectorizer_complain),
    ('lsa', lsa_complain)
])

In [ ]:
count_vectorizer_diagnosis = CountVectorizer()   
count_vectorizer_diagnosis.fit(X['Diagnosis in ED'])
lsa_diagnosis = TruncatedSVD(n_components=120)
lsa_diagnosis.fit(count_vectorizer_diagnosis.transform(X['Diagnosis in ED']))

pipe_diagnosis = Pipeline([                                             # text feature vectorizer and decomposition pipeline
    ('cvec', count_vectorizer_diagnosis),
    ('lsa', lsa_diagnosis)
])

In [ ]:
one_hot_encoder =  OneHotEncoder(drop='first')                          # one-hot encoder
one_hot_encoder.fit(X[feat_onehot])

OneHotEncoder(drop='first')

In [ ]:
standard_scaler_rest = StandardScaler()                                 # standard normalizing saler
standard_scaler_rest.fit(X[feat_scale_rest])

StandardScaler()

In [ ]:
pre_proc_cols = ColumnTransformer([                                     # Column transformer to conatenate the various pipelines
    ('impute_0', pipe_impute_0, feat_impute_0),
    ('impute_mean', pipe_impute_mean, feat_impute_mean),
    ('cv_complains', pipe_complain, 'Cheif_Complain_Translated'),
    ('cv_diagnosis', pipe_diagnosis, 'Diagnosis in ED'),
    ('one_hot', one_hot_encoder, feat_onehot),
    ('scale_rest', standard_scaler_rest, feat_scale_rest)
])

In [ ]:
# Traige Accuracy metric. 

def triage_accuracy(y, y_hat):    
  tru = 0
  for tr, pr in zip(y, y_hat):
    # the accuracy window is tunable, currently set for maximizing surety in exchange for increased window size
    if (pr - tr < 1.6 and pr - tr >= 0) or (tr - pr < 1.2 and tr - pr >= 0):
      tru+=1
  return tru/len(y)

from sklearn.metrics import make_scorer

triage_score = make_scorer(triage_accuracy)

In [ ]:
# helper function to map model's prediction to output class 

def y_class(y):
  if y < 1.4:
    return 1
  elif y < 2.4:
    return 2
  elif y < 3.4:
    return 3
  elif y < 4.4:
    return 4
  else:
    return 5

In [ ]:
# function to map model's prediction to output class 

def triage_output(y_hat):
  y_out = [y_class(y) for y in y_hat]
  return np.array(y_out)

In [ ]:
# a pseudo precision-recall-f1 score metric 

def triage_precision_recall_f1(y, y_hat):

  true_count_by_label = collections.Counter(y)
  
  y_out = triage_output(y_hat)
  pred_count_by_label = collections.Counter(y_out)

  correct_by_label = {
      1:0, 2:0, 3:0, 4:0, 5:0
  }

  # for tr, pr in zip(y, y_out):
  #   if tr == pr:
  #     correct_by_label[tr] += 1

  for tr, pr in zip(y, y_hat):
    if ( (tr + 0.6) > pr and (tr - 0.4) < pr ) :
      correct_by_label[tr] += 1
    elif ( (tr + 1.2) > pr and (tr - 0.8) < pr ) :
      correct_by_label[tr] += 0.4
    elif ( (tr + 1.5) > pr and (tr - 1) < pr ) :
      correct_by_label[tr] += 0.2
    elif ( pr > 5 and tr == 5 ):
      correct_by_label[5] += 1
    elif ( pr < 1 and tr == 1 ):
      correct_by_label[1] += 1

  precision = {
      1: correct_by_label[1] / pred_count_by_label[1],
      2: correct_by_label[2] / pred_count_by_label[2],
      3: correct_by_label[3] / pred_count_by_label[3],
      4: correct_by_label[4] / pred_count_by_label[4],
      5: 0,
  }

  recall = {
      1: correct_by_label[1] / true_count_by_label[1],
      2: correct_by_label[2] / true_count_by_label[2],
      3: correct_by_label[3] / true_count_by_label[3],
      4: correct_by_label[4] / true_count_by_label[4],
      5: 0,
  }

  f1 = {
      1: (2 * precision[1] * recall[1]) / (precision[1] + recall[1]),
      2: (2 * precision[2] * recall[2]) / (precision[2] + recall[2]),
      3: (2 * precision[3] * recall[3]) / (precision[3] + recall[3]),
      4: (2 * precision[4] * recall[4]) / (precision[4] + recall[4]),
      5: (2 * precision[5] * recall[5]) / (precision[5] + recall[5])
  }

  mean_val = {
      'mean precision': np.array(list(precision.values())).mean(),
      'mean recall': np.array(list(recall.values())).mean(),
      'mean f1': np.array(list(f1.values())).mean(),
  }

  return [precision, recall,  f1, mean_val]

In [ ]:
set_config(display="diagram")

Training the pipeline using various regression methods.

In [ ]:
from sklearn.linear_model import LinearRegression

lin = LinearRegression()

pipe_lin = Pipeline([
    ('prep_proc', pre_proc_cols),
    ('clf', lin)
])

lin_params = {}

lin_grid = GridSearchCV(pipe_lin, lin_params, cv=10, scoring=triage_score)

lin_grid.fit(X_train, y_train)
y_pred = lin_grid.predict(X_test)

print('Best Score:')
print(lin_grid.best_score_)
print('Best Params:')
print(lin_grid.best_params_)
print('Accuracy on test set:')
print(triage_score(lin_grid, X_test, y_test))
prec_recall_f1 = triage_precision_recall_f1(y_test, y_pred)
print('Classwise weak precision:')
print(prec_recall_f1[0])
print('Classwise weak recall:')
print(prec_recall_f1[1])
print('Classwise weak f1-score:')
print(prec_recall_f1[2])
print('Mean weak precision:')
print(prec_recall_f1[3]['mean precision'])
print('Mean weak recall:')
print(prec_recall_f1[3]['mean recall'])
print('Mean weak f1-score:')
print(prec_recall_f1[3]['mean f1'])

pipe_lin

Best Score:
0.9538375350140056
Best Params:
{}
Accuracy on test set:
0.9544364508393285
Classwise weak precision:
{1: 0.6, 2: 1.0749999999999997, 3: 0.8085365853658542, 4: 0.6054644808743174, 5: 0.32173913043478264}
Classwise weak recall:
{1: 0.38181818181818183, 2: 0.6615384615384613, 3: 0.7892857142857148, 4: 0.7337748344370867, 5: 0.3363636363636364}
Classwise weak f1-score:
{1: 0.4666666666666667, 2: 0.8190476190476187, 3: 0.798795180722892, 4: 0.663473053892216, 5: 0.3288888888888889}
Mean weak precision:
0.6821480393349908
Mean weak recall:
0.5805561656886162
Mean weak f1-score:
0.6153742818436564


Pipeline(steps=[('prep_proc',
                 ColumnTransformer(transformers=[('impute_0',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value=0,
                                                                                 strategy='constant')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['NRS_pain']),
                                                 ('impute_mean',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['SBP', 'DBP', 'HR', 'RR',
                                                   'BT', 'Saturation']),
                                                 ('cv_complains',
                                                  Pipeline(...
                                                                   TruncatedSVD(n_components=120))]),
                                                  'Cheif_Complain_Translated'),
                                                 ('cv_diagnosis',
                                                  Pipeline(steps=[('cvec',
                                                                   CountVectorizer()),
                                                                  ('lsa',
                                                                   TruncatedSVD(n_components=120))]),
                                                  'Diagnosis in ED'),
                                                 ('one_hot',
                                                  OneHotEncoder(drop='first'),
                                                  ['Group', 'Sex',
                                                   'Arrival mode', 'Injury',
                                                   'Pain']),
                                                 ('scale_rest',
                                                  StandardScaler(),
                                                  ['Age',
                                                   'Patients number per hour',
                                                   'Mental'])])),
                ('clf', LinearRegression())])

In [ ]:
from sklearn.linear_model import Ridge

rig = Ridge()

pipe_rig = Pipeline([
    ('prep_proc', pre_proc_cols),
    ('clf', rig)
])

rig_params = {}
rig_params['clf__alpha'] = [0.001, 0.01, 0.1, 1.0, 10]

rig_grid = GridSearchCV(pipe_rig, rig_params, cv=10, scoring=triage_score)
rig_grid.fit(X_train, y_train)

rig_res = pd.DataFrame(rig_grid.cv_results_)[['param_clf__alpha', 'mean_test_score']]
rig_res.to_csv('ridge_params_vs_score.csv')

print('Best Score:')
print(rig_grid.best_score_)
print('Best Params:')
print(rig_grid.best_params_)
print('Accuracy on test set:')
print(triage_score(rig_grid, X_test, y_test))

y_pred = rig_grid.predict(X_test)

prec_recall_f1 = triage_precision_recall_f1(y_test, y_pred)
print('Classwise weak precision:')
print(prec_recall_f1[0])
print('Classwise weak recall:')
print(prec_recall_f1[1])
print('Classwise weak f1-score:')
print(prec_recall_f1[2])
print('Mean weak precision:')
print(prec_recall_f1[3]['mean precision'])
print('Mean weak recall:')
print(prec_recall_f1[3]['mean recall'])
print('Mean weak f1-score:')
print(prec_recall_f1[3]['mean f1'])




Best Score:
0.9585434173669467
Best Params:
{'clf__alpha': 1.0}
Accuracy on test set:
0.9664268585131894
Classwise weak precision:
{1: 0.7333333333333334, 2: 1.1999999999999997, 3: 0.8294117647058828, 4: 0.6052356020942413, 5: 0.47142857142857153}
Classwise weak recall:
{1: 0.4, 2: 0.6646153846153844, 3: 0.8392857142857147, 4: 0.7655629139072854, 5: 0.30000000000000004}
Classwise weak f1-score:
{1: 0.5176470588235296, 2: 0.8554455445544552, 3: 0.8343195266272193, 4: 0.6760233918128661, 5: 0.36666666666666675}
Mean weak precision:
0.7678818543124057
Mean weak recall:
0.593892802561677
Mean weak f1-score:
0.6500204376969474


In [ ]:
from sklearn.linear_model import Lasso

las = Lasso()

pipe_las = Pipeline([
    ('prep_proc', pre_proc_cols),
    ('clf', las)
])

las_params = {}
las_params['clf__alpha'] = [0.001, 0.01, 0.1, 1.0, 10]

las_grid = GridSearchCV(pipe_las, las_params, cv=10, scoring=triage_score)
las_grid.fit(X_train, y_train)

las_res = pd.DataFrame(las_grid.cv_results_)[['param_clf__alpha', 'mean_test_score']]
las_res.to_csv('lasso_params_vs_score.csv')

print('Best Score:')
print(las_grid.best_score_)
print('Best Params:')
print(las_grid.best_params_)
print('Accuracy on test set:')
print(triage_score(las_grid, X_test, y_test))

y_pred = las_grid.predict(X_test)

prec_recall_f1 = triage_precision_recall_f1(y_test, y_pred)
print('Classwise weak precision:')
print(prec_recall_f1[0])
print('Classwise weak recall:')
print(prec_recall_f1[1])
print('Classwise weak f1-score:')
print(prec_recall_f1[2])
print('Mean weak precision:')
print(prec_recall_f1[3]['mean precision'])
print('Mean weak recall:')
print(prec_recall_f1[3]['mean recall'])
print('Mean weak f1-score:')
print(prec_recall_f1[3]['mean f1'])


Best Score:
0.9550420168067226
Best Params:
{'clf__alpha': 0.001}
Accuracy on test set:
0.9664268585131894
Classwise weak precision:
{1: 0.8285714285714286, 2: 1.413793103448275, 3: 0.8188888888888894, 4: 0.5863874345549744, 5: 0.5200000000000001}
Classwise weak recall:
{1: 0.5272727272727273, 2: 0.6307692307692303, 3: 0.8773809523809529, 4: 0.7417218543046367, 5: 0.23636363636363641}
Classwise weak f1-score:
{1: 0.6444444444444445, 2: 0.8723404255319143, 3: 0.8471264367816098, 4: 0.6549707602339189, 5: 0.32500000000000007}
Mean weak precision:
0.8335281710927136
Mean weak recall:
0.6027016802182367
Mean weak f1-score:
0.6687764133983776


In [ ]:
from sklearn.linear_model import SGDClassifier

sgd = SGDClassifier(max_iter=20000)

pipe_sgd = Pipeline([
    ('prep_proc', pre_proc_cols),
    ('clf', sgd)
])

sgd_params = {}

sgd_grid = GridSearchCV(pipe_sgd, sgd_params, cv=10, scoring=triage_score)
sgd_grid.fit(X_train, y_train)

print('Best Score:')
print(sgd_grid.best_score_)
print('Best Params:')
print(sgd_grid.best_params_)
print('Accuracy on test set:')
print(triage_score(sgd_grid, X_test, y_test))

y_pred = sgd_grid.predict(X_test)

prec_recall_f1 = triage_precision_recall_f1(y_test, y_pred)
print('Classwise weak precision:')
print(prec_recall_f1[0])
print('Classwise weak recall:')
print(prec_recall_f1[1])
print('Classwise weak f1-score:')
print(prec_recall_f1[2])
print('Mean weak precision:')
print(prec_recall_f1[3]['mean precision'])
print('Mean weak recall:')
print(prec_recall_f1[3]['mean recall'])
print('Mean weak f1-score:')
print(prec_recall_f1[3]['mean f1'])

Best Score:
0.9136554621848738
Best Params:
{}
Accuracy on test set:
0.9520383693045563
Classwise weak precision:
{1: 1.24, 2: 0.6049999999999998, 3: 0.6460674157303373, 4: 0.671830985915493, 5: 0.3333333333333333}
Classwise weak recall:
{1: 0.5636363636363636, 2: 0.7446153846153842, 3: 0.6845238095238098, 4: 0.6317880794701987, 5: 0.18181818181818182}
Classwise weak f1-score:
{1: 0.7749999999999999, 2: 0.6675862068965515, 3: 0.6647398843930639, 4: 0.6511945392491467, 5: 0.23529411764705885}
Mean weak precision:
0.6992463469958327
Mean weak recall:
0.5612763638127876
Mean weak f1-score:
0.5987629496371643


In [ ]:
from sklearn.svm import SVR

svr = SVR()

pipe_svr = Pipeline([
    ('prep_proc', pre_proc_cols),
    ('clf', svr)
])

svr_params = {}
svr_params['clf__kernel'] = [ 'poly', 'rbf', 'sigmoid']
svr_params['clf__C'] = [0.01, 0.1, 1.0, 10, 100]


svr_grid = GridSearchCV(pipe_svr, svr_params, cv=10, scoring=triage_score)
svr_grid.fit(X_train, y_train)

svr_res = pd.DataFrame(svr_grid.cv_results_)[['param_clf__C', 'param_clf__kernel', 'mean_test_score']]
svr_res.to_csv('SVR_params_vs_score.csv')

print('Best Score:')
print(svr_grid.best_score_)
print('Best Params:')
print(svr_grid.best_params_)
print('Accuracy on test set:')
print(triage_score(svr_grid, X_test, y_test))

y_pred = svr_grid.predict(X_test)

prec_recall_f1 = triage_precision_recall_f1(y_test, y_pred)
print('Classwise weak precision:')
print(prec_recall_f1[0])
print('Classwise weak recall:')
print(prec_recall_f1[1])
print('Classwise weak f1-score:')
print(prec_recall_f1[2])
print('Mean weak precision:')
print(prec_recall_f1[3]['mean precision'])
print('Mean weak recall:')
print(prec_recall_f1[3]['mean recall'])
print('Mean weak f1-score:')
print(prec_recall_f1[3]['mean f1'])

Best Score:
0.9515126050420168
Best Params:
{'clf__C': 10, 'clf__kernel': 'rbf'}
Accuracy on test set:
0.9496402877697842
Classwise weak precision:
{1: 1.0666666666666667, 2: 1.2342857142857135, 3: 0.7754189944134086, 4: 0.591666666666667, 5: 0.55}
Classwise weak recall:
{1: 0.29090909090909095, 2: 0.6646153846153843, 3: 0.8261904761904769, 4: 0.7523178807947024, 5: 0.2}
Classwise weak f1-score:
{1: 0.4571428571428572, 2: 0.8639999999999995, 3: 0.8000000000000007, 4: 0.6623906705539362, 5: 0.2933333333333334}
Mean weak precision:
0.8436076084064912
Mean weak recall:
0.546806566501931
Mean weak f1-score:
0.6153733722060254


In [ ]:
from sklearn.neighbors import KNeighborsRegressor

knn = KNeighborsRegressor()

pipe_knn = Pipeline([
    ('prep_proc', pre_proc_cols),
    ('clf', knn)
])

knn_params = {}
knn_params['clf__n_neighbors'] = [3, 5, 7]
knn_params['clf__weights'] = ['uniform', 'distance']

knn_grid = GridSearchCV(pipe_knn, knn_params, cv=10, scoring=triage_score)
knn_grid.fit(X_train, y_train)

knn_res = pd.DataFrame(knn_grid.cv_results_)[['param_clf__n_neighbors', 'param_clf__weights', 'mean_test_score']]
knn_res.to_csv('knn_params_vs_score.csv')

print('Best Score:')
print(knn_grid.best_score_)
print('Best Params:')
print(knn_grid.best_params_)
print('Accuracy on test set:')
print(triage_score(knn_grid, X_test, y_test))

y_pred = knn_grid.predict(X_test)

prec_recall_f1 = triage_precision_recall_f1(y_test, y_pred)
print('Classwise weak precision:')
print(prec_recall_f1[0])
print('Classwise weak recall:')
print(prec_recall_f1[1])
print('Classwise weak f1-score:')
print(prec_recall_f1[2])
print('Mean weak precision:')
print(prec_recall_f1[3]['mean precision'])
print('Mean weak recall:')
print(prec_recall_f1[3]['mean recall'])
print('Mean weak f1-score:')
print(prec_recall_f1[3]['mean f1'])

Best Score:
0.9349439775910364
Best Params:
{'clf__n_neighbors': 7, 'clf__weights': 'uniform'}
Accuracy on test set:
0.9424460431654677
Classwise weak precision:
{1: 1.35, 2: 2.0588235294117645, 3: 0.9687074829931979, 4: 0.4008064516129036, 5: 0.6000000000000001}
Classwise weak recall:
{1: 0.49090909090909096, 2: 0.5384615384615383, 3: 0.8476190476190482, 4: 0.6582781456953649, 5: 0.02727272727272728}
Classwise weak f1-score:
{1: 0.7200000000000001, 2: 0.8536585365853657, 3: 0.9041269841269847, 4: 0.49824561403508827, 5: 0.05217391304347827}
Mean weak precision:
1.0756674928035732
Mean weak recall:
0.5125081099915539
Mean weak f1-score:
0.6056410095581835


In [ ]:
from sklearn.cross_decomposition import PLSRegression

pls = PLSRegression()

pipe_pls = Pipeline([
    ('prep_proc', pre_proc_cols),
    ('clf', pls)
])

pls_params = {}
pls_params['clf__n_components'] = [1, 2, 5, 10, 15]

pls_grid = GridSearchCV(pipe_pls, pls_params, cv=10, scoring=triage_score)

pls_grid.fit(X_train, y_train)

print('Best Score:')
print(pls_grid.best_score_)
print('Best Params:')
print(pls_grid.best_params_)
print('Accuracy on test set:')
print(triage_score(pls_grid, X_test, y_test))

y_pred = pls_grid.predict(X_test)

prec_recall_f1 = triage_precision_recall_f1(y_test, y_pred)
print('Classwise weak precision:')
print(prec_recall_f1[0])
print('Classwise weak recall:')
print(prec_recall_f1[1])
print('Classwise weak f1-score:')
print(prec_recall_f1[2])
print('Mean weak precision:')
print(prec_recall_f1[3]['mean precision'])
print('Mean weak recall:')
print(prec_recall_f1[3]['mean recall'])
print('Mean weak f1-score:')
print(prec_recall_f1[3]['mean f1'])

Best Score:
0.9550420168067226
Best Params:
{'clf__n_components': 2}
Accuracy on test set:
0.9664268585131894
Classwise weak precision:
{1: 0.8400000000000001, 2: 1.2666666666666662, 3: 0.7817204301075272, 4: 0.5551912568306018, 5: 0.42000000000000004}
Classwise weak recall:
{1: 0.38181818181818183, 2: 0.6430769230769228, 3: 0.8654761904761908, 4: 0.6728476821192061, 5: 0.19090909090909092}
Classwise weak f1-score:
{1: 0.525, 2: 0.8530612244897956, 3: 0.8214689265536728, 4: 0.6083832335329349, 5: 0.2625}
Mean weak precision:
0.7727156707209591
Mean weak recall:
0.5508256136799184
Mean weak f1-score:
0.6140826769152807


In [ ]:
from sklearn.linear_model import BayesianRidge

brg = BayesianRidge()

pipe_brg = Pipeline([
    ('prep_proc', pre_proc_cols),
    ('clf', brg)
])

brg_params = {}

brg_grid = GridSearchCV(pipe_brg, brg_params, cv=10, scoring=triage_score)
brg_grid.fit(X_train, y_train)

print('Best Score:')
print(brg_grid.best_score_)
print('Best Params:')
print(brg_grid.best_params_)
print('Accuracy on test set:')
print(triage_score(brg_grid, X_test, y_test))

y_pred = brg_grid.predict(X_test)

prec_recall_f1 = triage_precision_recall_f1(y_test, y_pred)
print('Classwise weak precision:')
print(prec_recall_f1[0])
print('Classwise weak recall:')
print(prec_recall_f1[1])
print('Classwise weak f1-score:')
print(prec_recall_f1[2])
print('Mean weak precision:')
print(prec_recall_f1[3]['mean precision'])
print('Mean weak recall:')
print(prec_recall_f1[3]['mean recall'])
print('Mean weak f1-score:')
print(prec_recall_f1[3]['mean f1'])

Best Score:
0.9573949579831931
Best Params:
{}
Accuracy on test set:
0.9616306954436451
Classwise weak precision:
{1: 0.7714285714285715, 2: 1.699999999999999, 3: 0.8010989010989015, 4: 0.547524752475248, 5: 1.4000000000000001}
Classwise weak recall:
{1: 0.49090909090909096, 2: 0.6276923076923073, 3: 0.8678571428571432, 4: 0.7324503311258285, 5: 0.1272727272727273}
Classwise weak f1-score:
{1: 0.6, 2: 0.9168539325842693, 3: 0.8331428571428574, 4: 0.6266288951841364, 5: 0.23333333333333336}
Mean weak precision:
1.044010445000544
Mean weak recall:
0.5692363199714194
Mean weak f1-score:
0.6419918036489192


In [ ]:
from sklearn.tree import DecisionTreeRegressor

dtr = DecisionTreeRegressor()

pipe_dtr = Pipeline([
    ('prep_proc', pre_proc_cols),
    ('clf', dtr)
])

dtr_params = {}
dtr_params['clf__criterion'] = ['squared_error', 'friedman_mse']
dtr_params['clf__splitter'] = ['best', 'random']

dtr_grid = GridSearchCV(pipe_dtr, dtr_params, cv=10, scoring=triage_score)
dtr_grid.fit(X_train, y_train)

dtr_res = pd.DataFrame(dtr_grid.cv_results_)[['param_clf__splitter', 'param_clf__criterion', 'mean_test_score']]
dtr_res.to_csv('dtr_params_vs_score.csv')

print('Best Score:')
print(dtr_grid.best_score_)
print('Best Params:')
print(dtr_grid.best_params_)
print('Accuracy on test set:')
print(triage_score(dtr_grid, X_test, y_test))

y_pred = dtr_grid.predict(X_test)

prec_recall_f1 = triage_precision_recall_f1(y_test, y_pred)
print('Classwise weak precision:')
print(prec_recall_f1[0])
print('Classwise weak recall:')
print(prec_recall_f1[1])
print('Classwise weak f1-score:')
print(prec_recall_f1[2])
print('Mean weak precision:')
print(prec_recall_f1[3]['mean precision'])
print('Mean weak recall:')
print(prec_recall_f1[3]['mean recall'])
print('Mean weak f1-score:')
print(prec_recall_f1[3]['mean f1'])

Best Score:
0.8970868347338936
Best Params:
{'clf__criterion': 'squared_error', 'clf__splitter': 'best'}
Accuracy on test set:
0.894484412470024
Classwise weak precision:
{1: 0.96, 2: 0.4819277108433733, 3: 0.6588235294117649, 4: 0.6080536912751677, 5: 0.37037037037037035}
Classwise weak recall:
{1: 0.43636363636363634, 2: 0.6153846153846152, 3: 0.6000000000000002, 4: 0.6, 5: 0.45454545454545453}
Classwise weak f1-score:
{1: 0.6, 2: 0.5405405405405403, 3: 0.6280373831775703, 4: 0.604, 5: 0.40816326530612246}
Mean weak precision:
0.6158350603801352
Mean weak recall:
0.5412587412587413
Mean weak f1-score:
0.5561482378048466


In [ ]:
from sklearn.ensemble import RandomForestRegressor

rft = RandomForestRegressor()

pipe_rft = Pipeline([
    ('prep_proc', pre_proc_cols),
    ('clf', rft)
])

rft_params = {}
rft_params['clf__n_estimators'] = [20, 50, 80, 100, 150, 200]
rft_params['clf__max_depth'] = [None, 3, 10, 20, 30]

rft_grid = GridSearchCV(pipe_rft, rft_params, cv=10, scoring=triage_score)
rft_grid.fit(X_train, y_train)

rft_res = pd.DataFrame(rft_grid.cv_results_)[['param_clf__n_estimators', 'param_clf__max_depth', 'mean_test_score']]
rft_res.to_csv('rft_params_vs_score.csv')

print('Best Score:')
print(rft_grid.best_score_)
print('Best Params:')
print(rft_grid.best_params_)
print('Accuracy on test set:')
print(triage_score(rft_grid, X_test, y_test))

y_pred = rft_grid.predict(X_test)

prec_recall_f1 = triage_precision_recall_f1(y_test, y_pred)
print('Classwise weak precision:')
print(prec_recall_f1[0])
print('Classwise weak recall:')
print(prec_recall_f1[1])
print('Classwise weak f1-score:')
print(prec_recall_f1[2])
print('Mean weak precision:')
print(prec_recall_f1[3]['mean precision'])
print('Mean weak recall:')
print(prec_recall_f1[3]['mean recall'])
print('Mean weak f1-score:')
print(prec_recall_f1[3]['mean f1'])

Best Score:
0.9527030812324929
Best Params:
{'clf__max_depth': None, 'clf__n_estimators': 200}
Accuracy on test set:
0.9616306954436451
Classwise weak precision:
{1: 1.05, 2: 1.3214285714285707, 3: 0.7729729729729734, 4: 0.5628140703517595, 5: 1.4}
Classwise weak recall:
{1: 0.38181818181818183, 2: 0.5692307692307689, 3: 0.8511904761904767, 4: 0.7417218543046367, 5: 0.06363636363636363}
Classwise weak f1-score:
{1: 0.56, 2: 0.7956989247311822, 3: 0.8101983002832864, 4: 0.6400000000000008, 5: 0.1217391304347826}
Mean weak precision:
1.0214431229506609
Mean weak recall:
0.5215195290360854
Mean weak f1-score:
0.5855272710898504


In [ ]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor  # for base estimator

ada = AdaBoostRegressor()

pipe_ada = Pipeline([
    ('prep_proc', pre_proc_cols),
    ('clf', ada)
])

ada_params = {}
ada_params['clf__n_estimators'] = [50, 100, 150, 200]
ada_params['clf__base_estimator'] = [DecisionTreeRegressor(max_depth=3), 
                                     DecisionTreeRegressor(max_depth=10), 
                                     DecisionTreeRegressor(max_depth=20)]

ada_grid = GridSearchCV(pipe_ada, ada_params, cv=10, scoring=triage_score)
ada_grid.fit(X_train, y_train)

ada_res = pd.DataFrame(ada_grid.cv_results_)[['param_clf__n_estimators', 'param_clf__base_estimator', 'mean_test_score']]
ada_res.to_csv('ada_params_vs_score.csv')

print('Best Score:')
print(ada_grid.best_score_)
print('Best Params:')
print(ada_grid.best_params_)
print('Accuracy on test set:')
print(triage_score(ada_grid, X_test, y_test))

y_pred = ada_grid.predict(X_test)

prec_recall_f1 = triage_precision_recall_f1(y_test, y_pred)
print('Classwise weak precision:')
print(prec_recall_f1[0])
print('Classwise weak recall:')
print(prec_recall_f1[1])
print('Classwise weak f1-score:')
print(prec_recall_f1[2])
print('Mean weak precision:')
print(prec_recall_f1[3]['mean precision'])
print('Mean weak recall:')
print(prec_recall_f1[3]['mean recall'])
print('Mean weak f1-score:')
print(prec_recall_f1[3]['mean f1'])

Best Score:
0.9657282913165265
Best Params:
{'clf__base_estimator': DecisionTreeRegressor(max_depth=10), 'clf__n_estimators': 100}
Accuracy on test set:
0.9640287769784173
Classwise weak precision:
{1: 1.05, 2: 1.0874999999999992, 3: 0.69795918367347, 4: 0.6099447513812158, 5: 0.5}
Classwise weak recall:
{1: 0.38181818181818183, 2: 0.535384615384615, 3: 0.8142857142857151, 4: 0.73112582781457, 5: 0.09090909090909091}
Classwise weak f1-score:
{1: 0.56, 2: 0.7175257731958757, 3: 0.7516483516483522, 4: 0.6650602409638559, 5: 0.15384615384615385}
Mean weak precision:
0.7890807870109371
Mean weak recall:
0.5107046860424346
Mean weak f1-score:
0.5696161039308476


In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

gdb = GradientBoostingRegressor()

pipe_gdb = Pipeline([
    ('prep_proc', pre_proc_cols),
    ('clf', gdb)
])

gdb_params = {}
gdb_params['clf__learning_rate'] = [0.01, 0.1, 1]
gdb_params['clf__n_estimators'] = [50, 100, 150, 200]

gdb_grid = GridSearchCV(pipe_gdb, gdb_params, cv=10, scoring=triage_score)
gdb_grid.fit(X_train, y_train)

gdb_res = pd.DataFrame(gdb_grid.cv_results_)[['param_clf__n_estimators', 'param_clf__learning_rate', 'mean_test_score']]
gdb_res.to_csv('gdb_params_vs_score.csv')

print('Best Score:')
print(gdb_grid.best_score_)
print('Best Params:')
print(gdb_grid.best_params_)
print('Accuracy on test set:')
print(triage_score(gdb_grid, X_test, y_test))

y_pred = gdb_grid.predict(X_test)

prec_recall_f1 = triage_precision_recall_f1(y_test, y_pred)
print('Classwise weak precision:')
print(prec_recall_f1[0])
print('Classwise weak recall:')
print(prec_recall_f1[1])
print('Classwise weak f1-score:')
print(prec_recall_f1[2])
print('Mean weak precision:')
print(prec_recall_f1[3]['mean precision'])
print('Mean weak recall:')
print(prec_recall_f1[3]['mean recall'])
print('Mean weak f1-score:')
print(prec_recall_f1[3]['mean f1'])

Best Score:
0.9609523809523809
Best Params:
{'clf__learning_rate': 0.1, 'clf__n_estimators': 100}
Accuracy on test set:
0.9592326139088729
Classwise weak precision:
{1: 0.72, 2: 2.1444444444444426, 3: 0.7405940594059411, 4: 0.5989528795811523, 5: 2.5999999999999996}
Classwise weak recall:
{1: 0.32727272727272727, 2: 0.5938461538461534, 3: 0.8904761904761911, 4: 0.7576158940397356, 5: 0.11818181818181817}
Classwise weak f1-score:
{1: 0.4499999999999999, 2: 0.9301204819277101, 3: 0.8086486486486493, 4: 0.6690058479532167, 5: 0.2260869565217391}
Mean weak precision:
1.3607982766863072
Mean weak recall:
0.5374785567633251
Mean weak f1-score:
0.616772387010263


In [ ]:
from sklearn.neural_network import MLPRegressor

mlp = MLPRegressor(max_iter=1000)

pipe_mlp = Pipeline([
    ('prep_proc', pre_proc_cols),
    ('clf', mlp)
])

mlp_params = {}
mlp_params['clf__hidden_layer_sizes'] = [(10,10), (20,20), (20,10,10)]

mlp_grid = GridSearchCV(pipe_mlp, mlp_params, cv=10, scoring=triage_score)
mlp_grid.fit(X_train, y_train)

mlp_res = pd.DataFrame(mlp_grid.cv_results_)[['param_clf__hidden_layer_sizes', 'mean_test_score']]
mlp_res.to_csv('mlp_params_vs_score.csv')

print('Best Score:')
print(mlp_grid.best_score_)
print('Best Params:')
print(mlp_grid.best_params_)
print('Accuracy on test set:')
print(triage_score(mlp_grid, X_test, y_test))

y_pred = mlp_grid.predict(X_test)

prec_recall_f1 = triage_precision_recall_f1(y_test, y_pred)
print('Classwise weak precision:')
print(prec_recall_f1[0])
print('Classwise weak recall:')
print(prec_recall_f1[1])
print('Classwise weak f1-score:')
print(prec_recall_f1[2])
print('Mean weak precision:')
print(prec_recall_f1[3]['mean precision'])
print('Mean weak recall:')
print(prec_recall_f1[3]['mean recall'])
print('Mean weak f1-score:')
print(prec_recall_f1[3]['mean f1'])

Best Score:
0.9124789915966385
Best Params:
{'clf__hidden_layer_sizes': (20, 20)}
Accuracy on test set:
0.894484412470024
Classwise weak precision:
{1: 0.575, 2: 0.6852459016393441, 3: 0.7098765432098775, 4: 0.6309859154929582, 5: 0.16363636363636366}
Classwise weak recall:
{1: 0.41818181818181815, 2: 0.6430769230769229, 3: 0.6845238095238104, 4: 0.593377483443709, 5: 0.3272727272727273}
Classwise weak f1-score:
{1: 0.4842105263157894, 2: 0.6634920634920634, 3: 0.6969696969696978, 4: 0.6116040955631404, 5: 0.21818181818181823}
Mean weak precision:
0.5529489447957087
Mean weak recall:
0.5332865522997976
Mean weak f1-score:
0.5348916401045019
